In [ ]:
%pip install ultralytics requests opencv-python-headless

In [ ]:
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

import cv2
import matplotlib.pyplot as plt
import keras
import requests

Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.7/78.2 GB disk)


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="APIKEY")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
dataset.location

'/content/football-players-detection-1'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
ruta_drive = "/content/drive/MyDrive/IABD/PROIA/DeteccionImagen/"

Mounted at /content/drive


In [ ]:
import torch
model = YOLO("yolov8l.pt")

model.train(data=f'{dataset.location}/data.yaml', epochs=30, imgsz=640, plots=True)

In [ ]:
modelo2 = YOLO(ruta_drive + 'best1.pt')

In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Tuple, Optional, List, Dict, Any
import cv2
import numpy as np

@dataclass(frozen=True)
class Point:
    x: float
    y: float

    @property
    def int_xy_tuple(self) -> Tuple[int, int]:
        return int(self.x), int(self.y)


@dataclass(frozen=True)
class Rect:
    x: float
    y: float
    width: float
    height: float

    @property
    def min_x(self) -> float:
        return self.x

    @property
    def min_y(self) -> float:
        return self.y

    @property
    def max_x(self) -> float:
        return self.x + self.width

    @property
    def max_y(self) -> float:
        return self.y + self.height

    @property
    def top_left(self) -> Point:
        return Point(x=self.x, y=self.y)

    @property
    def bottom_right(self) -> Point:
        return Point(x=self.x + self.width, y=self.y + self.height)

    @property
    def bottom_center(self) -> Point:
        return Point(x=self.x + self.width / 2, y=self.y + self.height)

    @property
    def top_center(self) -> Point:
        return Point(x=self.x + self.width / 2, y=self.y)

    @property
    def center(self) -> Point:
        return Point(x=self.x + self.width / 2, y=self.y + self.height / 2)



@dataclass(frozen=True)
class Color:
    r: int
    g: int
    b: int

    @property
    def bgr_tuple(self) -> Tuple[int, int, int]:
        return self.b, self.g, self.r

    @classmethod
    def from_hex_string(cls, hex_string: str) -> Color:
        r, g, b = tuple(int(hex_string[1 + i:1 + i + 2], 16) for i in (0, 2, 4))
        return Color(r=r, g=g, b=b)



def draw_ellipse(image: np.ndarray, rect: Rect, color: Color, thickness: int = 2) -> np.ndarray:
    cv2.ellipse(
        image,
        center=rect.bottom_center.int_xy_tuple,
        axes=(int(rect.width), int(0.35 * rect.width)),
        angle=0.0,
        startAngle=-45,
        endAngle=235,
        color=color.bgr_tuple,
        thickness=thickness,
        lineType=cv2.LINE_4
    )
    return image

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture(ruta_drive + '/bundesliga.mp4')

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(ruta_drive + '/prediccionBundesligaa.mp4', fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break
    results = modelo2.predict(frame)

    # Dibujar elipses alrededor de los objetos detectados
    for result in results[0].boxes:
        x1, y1, x2, y2 = result.xyxy[0]
        rect = Rect(x1, y1, x2 - x1, y2 - y1)
        color = Color(255, 255, 255)
        frame = draw_ellipse(frame, rect, color)

        class_name = modelo2.names[int(result.cls[0])]
        if class_name == 'referee':
          x1, y1, x2, y2 = result.xyxy[0]
          rect = Rect(x1, y1, x2 - x1, y2 - y1)
          color = Color(0, 0, 0)
          frame = draw_ellipse(frame, rect, color)

          text_x = x1
          text_y = y1 - 20
          cv2.putText(frame, class_name, (int(text_x), int(text_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.32, (0, 0, 0), 2)
        else:
          text_x = x1
          text_y = y1 - 20

          cv2.putText(frame, class_name, (int(text_x), int(text_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.32, (255, 255, 255), 2)

    out.write(frame)

cap.release()
out.release()